<a href="https://colab.research.google.com/github/NiceSoyce/DS_Lab2/blob/main/robert_thibault_lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analyse en Composantes Principales (ACP)

## Partie 1 : Analyse du dataset *decathlon*

### 1. Préparation des donneés

### 2. Initialisation du ACP

### 3. Analyse de l'inertie

### 4. Étude des variables

### 5. Étude des variables

### 6. Qualité de représentation

### 7. Synthèse de résultats

## Partie 2 : Analyse du dataset *digits*

### 1. Chargement et visualisation des données

### 2. Prétraitement

### 3. Classification sans ACP

### 4. Réduction de dimension avec ACP

### 5. Classification après ACP

### 6. Visualisation